In [1]:
%%time

import pandas as pd 
import scanpy as sc
import phenograph
import parc
from utag import utag
import os 
import numpy as np
import time
import anndata as ad
import scanpy.external as sce
import igraph
#import rapids_singlecell as rsc
os.chdir("/home/abombin/Mawa/multiplex-analysis-web-apps")

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 5.72 s, sys: 3.14 s, total: 8.87 s
Wall time: 7.27 s


In [ ]:
%%time
adata = sc.read("input/clust_dat.h5ad")
print(adata)


In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, metric='euclidean', n_pcs=None, random_state=42)
sc.tl.leiden(adata, resolution=1, random_state=42, n_iterations=-1, flavor="igraph")
print(adata.obsp['connectivities'])
print(adata.uns['neighbors'])

neighb = adata.uns['neighbors'][0]

In [ ]:
communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="leiden", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

print(graph)

cur_graph  = graph['edges']
cur_graph

In [ ]:
print(type(graph))
print(type(adata.obsp['connectivities']))

In [ ]:
adata = sc.read("input/clust_dat.h5ad")
sc.tl.pca(adata)
sce.tl.phenograph(adata, clustering_algo="leiden", k=30)
print(adata)


In [ ]:
print(adata.obsp)
#print(adata.uns['neighbors'])

In [5]:
%%time

adata = sc.read("input/clust_dat.h5ad")
# Assuming adata is your AnnData object and it has been preprocessed


CPU times: user 97.6 ms, sys: 54.3 ms, total: 152 ms
Wall time: 150 ms


/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
%%time

communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="leiden", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

adata.obsp['connectivities'] = graph
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'n_neighbors': 10, 'method': 'phenograph', 'random_state': 42, 'metric': 'euclidean'}}

Finding 10 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1.3123178482055664 seconds
Jaccard graph constructed in 1.5891451835632324 seconds
Running Leiden optimization
Leiden completed in 10.359914302825928 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 13.80940556526184 seconds
CPU times: user 21.3 s, sys: 571 ms, total: 21.9 s
Wall time: 13.8 s


In [7]:
%%time
sc.tl.umap(adata)

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/umap/spectral.py:485: RuntimeWarning: divide by zero encountered in divide
  1.0 / sqrt_deg, 0, graph.shape[0], graph.shape[0]


CPU times: user 1min 32s, sys: 2min 14s, total: 3min 46s
Wall time: 18 s


In [2]:
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
adata.layers["counts"] = adata.X.copy()

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
%%time

communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="louvian", k=10, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="brute",
                                               seed=42, n_iterations=-1, n_jobs=8)

adata.obsp['connectivities'] = graph
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'n_neighbors': 10, 'method': 'phenograph', 'random_state': 42, 'metric': 'euclidean'}}

Finding 10 nearest neighbors using minkowski metric and 'auto' algorithm


KeyboardInterrupt: 

In [2]:
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
adata.raw = adata
adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.tl.pca(adata, n_comps=10)

print(adata.raw.X)

print("____")
print(adata.X)

print("___")

print(adata.layers["counts"])

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


[[4.06493182 0.61428084 0.08139293 ... 0.41146035 1.98478415 4.65075001]
 [3.71947915 0.05329087 0.10381342 ... 0.82665942 1.54604683 4.84629856]
 [3.62346994 0.43748003 0.05921958 ... 2.33795683 1.90686261 4.70846475]
 ...
 [4.47138238 0.70963534 1.01931117 ... 1.36353363 4.05493    4.26338494]
 [4.09397373 0.57299476 1.07635983 ... 1.58267492 2.86271018 3.93975399]
 [3.99236226 0.58628336 1.19875082 ... 1.37660655 3.92141449 4.28453506]]
____
[[4.06493182 0.61428084 0.08139293 ... 0.41146035 1.98478415 4.65075001]
 [3.71947915 0.05329087 0.10381342 ... 0.82665942 1.54604683 4.84629856]
 [3.62346994 0.43748003 0.05921958 ... 2.33795683 1.90686261 4.70846475]
 ...
 [4.47138238 0.70963534 1.01931117 ... 1.36353363 4.05493    4.26338494]
 [4.09397373 0.57299476 1.07635983 ... 1.58267492 2.86271018 3.93975399]
 [3.99236226 0.58628336 1.19875082 ... 1.37660655 3.92141449 4.28453506]]
___
[[4.821430e+01 7.143000e-01 7.140000e-02 ... 4.286000e-01 5.285700e+00
  8.728570e+01]
 [5.072970e+01 6

In [3]:
communities, graph, Q = phenograph.cluster(adata.obsm['X_pca'], clustering_algo="louvian", k=30, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 400.9026267528534 seconds
Jaccard graph constructed in 58.21604633331299 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 462.5701289176941 seconds


In [5]:
adata.obsp['connectivities'] = graph
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'n_neighbors': 10, 'method': 'phenograph', 'random_state': 42, 'metric': 'euclidean'}}

In [6]:
%%time
sc.tl.umap(adata)

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/umap/spectral.py:485: RuntimeWarning: divide by zero encountered in divide
  1.0 / sqrt_deg, 0, graph.shape[0], graph.shape[0]


CPU times: user 9h 50min 30s, sys: 1h 50min 10s, total: 11h 40min 41s
Wall time: 1h 3min 26s


In [3]:
%%time
import umap
cur_umap_1 = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(mat)



: 

In [ ]:
%%time
import umap
cur_umap_1 = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(adata.layers["counts"])

In [ ]:
%%time
import umap
cur_umap = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(adata.obsm['X_pca'])

KeyboardInterrupt: 